In [13]:
import torch
import torch.nn
import numpy as np
import pandas as pd
from torcheval.metrics.functional import bleu_score
import random
from recipe_nlg import RecipeNLGDataset
from models import NextByteTransformer
from Save_Results import save_bleu
import kagglehub

def get_bleu(model, num_prompts_per_trial, dataset):
    
    pred_recipes, ground_truth = [], []
    
    for i in range(num_prompts_per_trial):
        
        # random index for a random recipe
        rand_index = random.randint(0, dataset.__len__()-1)
        # get the title on its own from the cleaned data frame
        title_prompt = dataset.recipes.iloc[rand_index]['title']
        # get the full recipe string (not tokenized)
        recipe_ref = dataset.recipe_strings.iloc[rand_index]
        # pred = model.generate(title_prompt) # replace with actual generation method
        pred = "hamburger blah blah blah blah and the guy said hello whats up man"
        pred_recipes.append(pred)
        ground_truth.append(recipe_ref)
        
    bleu = bleu_score(input=pred_recipes, target=ground_truth, n_gram=4) # n_gram = max count of sequential words to compare strings on
    
    # bleu is a 1d tensor, extract the value and convert to an np.float64 first
    return bleu.item()


# same params used in all training
context_length = 512
d_model = 512
num_heads = 8
num_hidden_layers = 8
d_hidden = 2048
num_decoders = 2
num_epochs = 8

# instantiate plain model
model = NextByteTransformer(
    vocab_size=20000,
    context_length=context_length,
    d_model=d_model,
    num_heads=num_heads,
    num_hidden_layers=num_hidden_layers,
    d_hidden=d_hidden,
    num_decoders=num_decoders
) 

# LOAD MODEL HERE
model_name = ""
# state_dict_path = ""
# model.load_state_dict(state_dict_path)

In [5]:
path = kagglehub.dataset_download("paultimothymooney/recipenlg")
# Load the dataset
df = pd.read_csv(path + "/RecipeNLG_dataset.csv", header=0)
dataset = RecipeNLGDataset(df=df, mode='all')

In [11]:
num_prompts_per_trial = 10
num_trials = 10

results = ['trial', 'bleu_score']


for i in range(num_trials):
    results.append([i + 1, get_bleu(model, num_prompts_per_trial, dataset)])
    
print(results)

['trial', 'bleu_score', [1, 0.0], [2, 0.0], [3, 0.0], [4, 0.0], [5, 0.0], [6, 0.0], [7, 0.0], [8, 0.0], [9, 0.0], [10, 0.0]]


In [ ]:
save_bleu(results=results, model_name=model_name)